## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,29.79,86,90,3.44,light snow
1,1,Albany,US,42.6001,-73.9662,31.08,89,100,1.92,light snow
2,2,Port Hedland,AU,-20.3167,118.5667,95.05,55,4,12.66,clear sky
3,3,Castro,BR,-24.7911,-50.0119,48.97,99,12,4.79,few clouds
4,4,Auray,FR,47.6703,-2.9918,47.75,83,82,9.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter minimum desired temperature:"))
max_temp = float(input("Please enter maximum desired temperature:"))

Please enter minimum desired temperature:75
Please enter maximum desired temperature:90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Ihosy,MG,-22.4000,46.1167,75.72,56,21,7.36,few clouds
14,14,Hithadhoo,MV,-0.6000,73.0833,82.08,75,97,10.02,overcast clouds
18,18,Cayenne,GF,4.9333,-52.3333,77.09,90,98,1.99,overcast clouds
20,20,Cap Malheureux,MU,-19.9842,57.6142,83.46,61,15,7.00,few clouds
21,21,Hilo,US,19.7297,-155.0900,76.28,38,90,5.75,light rain
...,...,...,...,...,...,...,...,...,...,...
683,683,Byron Bay,AU,-28.6500,153.6167,87.91,70,1,11.81,clear sky
684,684,Kirakira,SB,-10.4544,161.9205,84.58,76,97,6.58,overcast clouds
687,687,Auki,SB,-8.7676,160.7034,82.83,86,92,6.38,overcast clouds
688,688,Bang Saphan,TH,11.2126,99.5117,85.95,63,57,12.37,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities[preferred_cities.isnull().any(axis=1)]

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
212,212,Aranos,NaN,-24.1333,19.1167,75.42,48,20,18.88,few clouds
243,243,Mariental,NaN,-24.6333,17.9667,75.20,28,18,4.32,few clouds


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. 
# Note: no empty rows, cities 212, 243, 313, 413 are all in Namibia. Will replace NaN with NA for Namibia's actual 2-letter code
clean_df = preferred_cities.fillna('NA')
clean_df.count()

Unnamed: 0             174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   174
Country                174
Max Temp               174
Current Description    174
Lat                    174
Lng                    174
Hotel Name             174
dtype: int64

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0].get('name')
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df[hotel_df.isnull().any(axis=1)]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
48,Ligayan,PH,81.99,overcast clouds,4.6533,119.4706,NaN
149,Mbekenyera,TZ,76.44,few clouds,-10.0000,38.9833,NaN
154,Latung,PH,83.05,overcast clouds,5.5000,120.8833,NaN
201,Simbahan,PH,82.15,overcast clouds,6.3032,120.5836,NaN
312,Tsaratanana,MG,80.47,broken clouds,-16.7833,47.6500,NaN
436,Gamba,GA,76.84,light rain,-2.6500,10.0000,NaN
478,Balaipungut,ID,86.41,overcast clouds,1.0500,101.2833,NaN
481,Lagos,NG,77.32,broken clouds,6.5833,3.7500,NaN
508,Mtambile,TZ,79.84,light rain,-5.3833,39.7000,NaN
582,Beloha,MG,86.92,few clouds,-25.1667,45.0500,NaN


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Ihosy,MG,75.72,few clouds,-22.4000,46.1167,Hotely Sariaka
14,Hithadhoo,MV,82.08,overcast clouds,-0.6000,73.0833,Scoop Guest House
18,Cayenne,GF,77.09,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
20,Cap Malheureux,MU,83.46,few clouds,-19.9842,57.6142,Pereybere Hotel & Spa
21,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
683,Byron Bay,AU,87.91,clear sky,-28.6500,153.6167,Backpackers Inn
684,Kirakira,SB,84.58,overcast clouds,-10.4544,161.9205,SanBiz Lodge
687,Auki,SB,82.83,overcast clouds,-8.7676,160.7034,Auki Motel
688,Bang Saphan,TH,85.95,broken clouds,11.2126,99.5117,Ladawan Boutique Hotel


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#max_temp = hotel_df["Max Temp"]

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))